In [1]:
import pandas as pd
import psycopg2
import warnings

In [2]:
warnings.simplefilter(action='ignore', category=UserWarning)

db_params = {
    'database': 'postgres',
    'user': 'postgres',
    'password': 'admin',
    'host': '127.0.0.1',
    'port': '5432'
}
conn = psycopg2.connect(**db_params)
conn.autocommit = True

### Задание 1.1
Найти десять самых продаваемых товаров по магазинам

In [3]:
sql_query = '''
WITH RankedProducts AS (
    SELECT
        products.Наименование AS product_name,
        sales.Номенклатура,
        branches.Ссылка AS branch_key,
        branches.Наименование AS branch_name,
        ROW_NUMBER() OVER (PARTITION BY branches.Ссылка ORDER BY sales.Номенклатура DESC) AS ranking
    FROM sales
    INNER JOIN products 
        ON sales.Номенклатура = products.Ссылка
    INNER JOIN branches 
        ON sales.Филиал = branches.Ссылка
    WHERE (UPPER(products.Наименование) NOT LIKE UPPER('%Доставка%') AND UPPER(products.Наименование) NOT LIKE UPPER('%Обработка%'))
    ),
    branches_new AS (
        SELECT *,
        CASE WHEN branches.Наименование LIKE '%склад%' OR branches.Наименование LIKE '%Склад%' THEN TRUE ELSE FALSE END AS is_storage
        FROM branches
    ),
    branches_with_products AS (
        SELECT 
            branches_new.Ссылка AS branch_key,
            branches_new.Наименование AS branch_name,
            ARRAY_AGG(RankedProducts.product_name) AS top_products
        FROM branches_new
        INNER JOIN RankedProducts
            ON RankedProducts.branch_key = branches_new.Ссылка
        WHERE RankedProducts.ranking BETWEEN 1 AND 10
        GROUP BY branches_new.Ссылка, branches_new.Наименование
    )
    SELECT branch_name, top_products FROM branches_with_products WHERE top_products IS NOT NULL;
'''

# Выполнение запроса и чтение результатов в DataFrame
df = pd.read_sql_query(sql_query, conn)

cursor = conn.cursor()

# Удаление таблицы, если она уже существует
cursor.execute("DROP TABLE IF EXISTS branch_products")

# Создание таблицы branch_products
cursor.execute('''
    CREATE TABLE branch_products (
        branch_name VARCHAR(255),
        top_products TEXT[]
    )
''')

# Вставка данных из DataFrame в таблицу branch_products
for index, row in df.iterrows():
    cursor.execute("INSERT INTO branch_products (branch_name, top_products) VALUES (%s, %s)", (row['branch_name'], row['top_products']))

conn.commit()
display(df)

,branch_name,top_products
0,ЕКБ Вайнера,[Память Trans Flash (microSDHC) 8 Gb class 10...
1,Самара ТЦ Аврора SMART,[Память Trans Flash (microSDHC) 8 Gb class 10...
2,Кра на Спутнике,[Память Trans Flash (microSDHC) 8 Gb class 10...
3,Новокуйбышевск пр. Победы,[Память Trans Flash (microSDHC) 8 Gb class 10...
4,Чита ТЦ Парад,[Память Trans Flash (microSDHC) 8 Gb class 10...
...,...,...
273,Ульяновск Гончарова,[Память Trans Flash (microSDHC) 8 Gb class 10...
274,яПрокопьевск на Шахтеров,[Память Trans Flash (microSDHC) 8 Gb class 10...
275,яХаб ИЗОТОП,[Память Trans Flash (microSDHC) 8 Gb class 10...
276,яХаб Партнер,[Память Trans Flash (microSDHC) 8 Gb class 10...


### Задание 1.2
Найти топ десять магазинов по суммарному количеству продаж

In [4]:
sql_query = '''
    CREATE TEMPORARY TABLE branches_new AS (
        SELECT *,
            CASE WHEN Наименование LIKE '%склад%' OR Наименование LIKE '%Склад%' THEN TRUE ELSE FALSE END AS is_storage
        FROM branches);
'''
cursor = conn.cursor()
cursor.execute(sql_query)

sql_query = '''
WITH RankedShops AS (
    SELECT *,
           CASE WHEN Наименование LIKE '%склад%' OR Наименование LIKE '%Склад%' THEN TRUE ELSE FALSE END AS is_storage
    FROM branches
    )
    SELECT
        branches_new.Наименование AS branch_name,
        SUM(sales.Количество) AS sum_of_sales
    FROM sales
    INNER JOIN branches_new ON branches_new.Ссылка = sales.Филиал
    WHERE is_storage = FALSE
    GROUP BY branch_name
    ORDER BY sum_of_sales DESC
    LIMIT 10
    OFFSET 1;
'''

df = pd.read_sql_query(sql_query, conn)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS top_shops")

# Создание таблицы top_shops
cursor.execute('''
    CREATE TABLE top_shops (
        branch_name VARCHAR(255),
        sum_of_sales NUMERIC
    )
''')

# Вставка данных из DataFrame в таблицу top_shops
for index, row in df.iterrows():
    cursor.execute("INSERT INTO top_shops (branch_name, sum_of_sales) VALUES (%s, %s)", (row['branch_name'], row['sum_of_sales']))
    
conn.commit()
display(df)

,branch_name,sum_of_sales
0,Новосиб Ватутина,94624.0
1,Ростов Красноармейская,79159.0
2,яВладивосток Луговая МБТ,74879.0
3,Омск на Гагарина,70353.0
4,яКомсомольск Аллея Труда,66382.0
5,ЕКБ Вайнера,63229.0
6,яН.Новгород Пл. Революции,63049.0
7,яСаратов Университетская,60728.0
8,Ростов Вавилония,60458.0
9,Краснодар ТЦ Кавказ,60359.0


### Задание 1.3
Вывести рейтинг товаров согласно суммарному количеству проданного товара за всю историю наблюдений со средним количеством продаж за день по убыванию

In [5]:
sql_query = '''
WITH products_ranking AS (
    SELECT
        products.Ссылка,
        products.Наименование AS product_name,
        SUM(sales.Количество) AS sum_of_sales,
        COUNT(DISTINCT DATE_TRUNC('day', sales.Период)) AS days_count,
        SUM(sales.Количество) / COUNT(DISTINCT DATE_TRUNC('day', sales.Период)) AS avg_sales,
        ROW_NUMBER() OVER (ORDER BY SUM(sales.Количество) DESC) AS ranking
    FROM sales
    INNER JOIN products
        ON sales.Номенклатура = products.Ссылка
    GROUP BY products.Ссылка
    )
    SELECT
        product_name,
        sum_of_sales,
        avg_sales
    FROM products_ranking
    ORDER BY ranking;
'''

df = pd.read_sql_query(sql_query, conn)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS products_ranking")

# Создание таблицы products_ranking
cursor.execute('''
    CREATE TABLE products_ranking (
        product_name VARCHAR(255),
        sum_of_sales NUMERIC,
        avg_sales NUMERIC
    )
''')

# Вставка данных из DataFrame в таблицу products_ranking
for index, row in df.iterrows():
    cursor.execute("INSERT INTO products_ranking (product_name, sum_of_sales, avg_sales) VALUES (%s, %s, %s)", (row['product_name'], row['sum_of_sales'], row['avg_sales']))
    
conn.commit()
display(df)

,product_name,sum_of_sales,avg_sales
0,Доставка внутри региона,1036108.0,9088.666667
1,"Разъем Noname [RJ45, 8P8C, кат. 5e, 1 шт]",256369.0,2136.408333
2,"Кабель UTP, 4 пары, одножильный (solid), кат. ...",212994.0,1774.950000
3,Обработка грузов на РРЦ,171546.0,3500.938776
4,Доставка от поставщика,43338.0,498.137931
...,...,...,...
19841,Сотовый телефон Samsung GT-S5330 WAVE 533 Bla...,-2.0,-0.500000
19842,"Монитор LG 21.5"" Flatron W2246S [1920x1080, D...",-2.0,-1.000000
19843,"Монитор LG LCD 20"" Flatron W2046S [1600x900, D...",-2.0,-1.000000
19844,"Монитор LG LCD 19"" Flatron W1934S/S-PF [1440x9...",-3.0,-1.000000


### Задание 1.4
Найти два лучших филиала согласно суммарному количеству проданного товара за всю историю наблюдений в регионе Урал по городу Екатеринбург с суммой продаж за январь

In [6]:
sql_query = '''
    SELECT
        branches.Наименование AS branch_name,
        SUM(sales.Количество) AS sum_of_sales
    FROM sales
    INNER JOIN branches
        ON branches.Ссылка = sales.Филиал
    WHERE 
        branches.Город = (
            SELECT Ссылка 
            FROM cities 
            WHERE Наименование = 'Екатеринбург'
        )
        AND EXTRACT(MONTH FROM Период) = 1
    GROUP BY branch_name
    ORDER BY sum_of_sales DESC
    LIMIT 2;
'''

df = pd.read_sql_query(sql_query, conn)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS top_ural_shops")

# Создание таблицы top_ural_shops
cursor.execute('''
    CREATE TABLE top_ural_shops (
        branch_name VARCHAR(255),
        sum_of_sales NUMERIC
    )
''')

# Вставка данных из DataFrame в таблицу top_ural_shops
for index, row in df.iterrows():
    cursor.execute("INSERT INTO top_ural_shops (branch_name, sum_of_sales) VALUES (%s, %s)", (row['branch_name'], row['sum_of_sales']))
    
conn.commit()
display(df)

,branch_name,sum_of_sales
0,ЕКБ старый cклад,42489.0
1,ЕКБ Вайнера,15605.0


### Задание 1.5
Рассчитать и вывести в какие часы и в какой день недели происходит максимальное количество продаж

In [7]:
sql_query = '''
    SELECT
        EXTRACT(DOW FROM Период) AS day_of_week,
        EXTRACT(HOUR FROM Период) AS hour,
        COUNT(*) AS sales_count
    FROM sales
    GROUP BY day_of_week, hour
    ORDER BY sales_count DESC
    LIMIT 3;
'''

df = pd.read_sql_query(sql_query, conn)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS sales_best_days")

# Создание таблицы sales_best_days
cursor.execute('''
    CREATE TABLE sales_best_days (
        day_of_week NUMERIC,
        hour NUMERIC,
        sales_count NUMERIC
    )
''')

# Вставка данных из DataFrame в таблицу sales_best_days
for index, row in df.iterrows():
    cursor.execute("INSERT INTO sales_best_days (day_of_week, hour, sales_count) VALUES (%s, %s, %s)", (row['day_of_week'], row['hour'], row['sales_count']))
    
conn.commit()
display(df)

,day_of_week,hour,sales_count
0,6.0,14.0,153267
1,6.0,15.0,150048
2,6.0,13.0,147808


In [8]:
cursor.close()
conn.close()